In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_data, upload_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get a dataframe of the transformed ethereum OHLC data we made in 02_ohlc_feature_target_engineering.ipynb

In [11]:
eth_ohlc = pd.read_csv('../data/transformed/ohlc/ethereum.csv')
eth_ohlc

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_percent_increase_to_avg_high_low,tmw_0_0_percent_increase_binary,...,last_100_day_70th_to_80th_pct_spread_count,last_100_day_80th_to_90th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_70th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count,last_100_day_95th_pct_spread_count
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.033594,1.0,...,18.0,9.0,18.0,94.0,86.0,79.0,73.0,55.0,46.0,28.0
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,0.097955,1.0,...,18.0,9.0,18.0,94.0,86.0,78.0,72.0,54.0,45.0,27.0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,-0.009901,0.0,...,18.0,10.0,18.0,94.0,86.0,78.0,72.0,54.0,44.0,26.0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,-0.007323,0.0,...,18.0,10.0,17.0,94.0,86.0,78.0,71.0,53.0,43.0,26.0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,-0.015908,0.0,...,18.0,10.0,17.0,94.0,86.0,78.0,70.0,52.0,42.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,2024-05-08,3013.0000,3036.0000,2944.0000,2975.0000,1.020000e+10,3400618.0,3.675000e+11,0.023697,1.0,...,8.0,4.0,3.0,47.0,33.0,25.0,15.0,7.0,3.0,0.0
3079,2024-05-09,2977.0000,3055.0000,2958.0000,3036.0000,9.600000e+09,3190231.0,3.670000e+11,-0.018445,0.0,...,8.0,4.0,3.0,47.0,33.0,25.0,15.0,7.0,3.0,0.0
3080,2024-05-10,3035.0000,3050.0000,2889.0000,2910.0000,1.060000e+10,3557434.0,3.650000e+11,0.004467,1.0,...,8.0,4.0,3.0,48.0,34.0,25.0,15.0,7.0,3.0,0.0
3081,2024-05-11,2909.0000,2935.0000,2894.0000,2911.0000,5.900000e+09,2033385.0,3.569000e+11,0.010306,1.0,...,8.0,4.0,3.0,48.0,34.0,25.0,15.0,7.0,3.0,0.0


### Upload this df to hopsworks feature store

In [12]:
# hopsworks requires datetime, when reading back from .csv, the datetime designation apparently goes away, along with nonnullable ints
eth_ohlc['date'] = pd.to_datetime(eth_ohlc['date'])

# Convert response binary columns to nullable integer type
eth_ohlc['tmw_1_0_percent_increase_binary'] = eth_ohlc['tmw_1_0_percent_increase_binary'].astype('Int64')


predictors_96 = ['rsi_2', 'volume_rsi_2', 'spread_rsi_2', 'last_2_day_80th_pct_volume_change_count', 
                 'last_2_day_60th_to_70th_pct_spread_count', 'last_2_day_40th_pct_spread_count', 'rsi_5', 'volume_rsi_5', 
                 'spread_rsi_5', 'last_5_day_60th_pct_price_change_count', 'last_5_day_70th_pct_price_change_count', 'rsi_10', 
                 'volume_rsi_10', 'spread_rsi_10', 'last_10_day_40th_to_50th_pct_price_change_count', 
                 'last_10_day_60th_pct_price_change_count', 'last_10_day_70th_to_80th_pct_volume_change_count', 
                 'last_10_day_80th_to_90th_pct_spread_count', 'last_10_day_40th_pct_spread_count', 'last_10_day_50th_pct_spread_count', 
                 'last_10_day_70th_pct_spread_count', 'last_10_day_80th_pct_spread_count', 'rsi_25', 'volume_rsi_25', 'spread_rsi_25', 
                 'last_25_day_40th_to_50th_pct_price_change_count', 'last_25_day_70th_to_80th_pct_price_change_count', 
                 'last_25_day_80th_to_90th_pct_price_change_count', 'last_25_day_40th_pct_price_change_count', 
                 'last_25_day_50th_pct_price_change_count', 'last_25_day_80th_pct_price_change_count', 
                 'last_25_day_40th_to_50th_pct_volume_change_count', 'last_25_day_50th_to_60th_pct_volume_change_count', 
                 'last_25_day_60th_to_70th_pct_volume_change_count', 'last_25_day_70th_to_80th_pct_volume_change_count', 
                 'last_25_day_80th_to_90th_pct_volume_change_count', 'last_25_day_50th_pct_volume_change_count', 
                 'last_25_day_60th_pct_volume_change_count', 'last_25_day_70th_pct_volume_change_count', 
                 'last_25_day_80th_pct_volume_change_count', 'last_25_day_40th_to_50th_pct_spread_count', 
                 'last_25_day_50th_to_60th_pct_spread_count', 'last_25_day_60th_to_70th_pct_spread_count', 
                 'last_25_day_70th_to_80th_pct_spread_count', 'last_25_day_80th_to_90th_pct_spread_count', 
                 'last_25_day_40th_pct_spread_count', 'last_25_day_60th_pct_spread_count', 'last_25_day_70th_pct_spread_count', 
                 'last_25_day_90th_pct_spread_count', 'rsi_50', 'last_50_day_50th_to_60th_pct_price_change_count', 
                 'last_50_day_60th_to_70th_pct_price_change_count', 'last_50_day_70th_to_80th_pct_price_change_count', 
                 'last_50_day_40th_pct_price_change_count', 'last_50_day_50th_pct_price_change_count', 
                 'last_50_day_70th_pct_price_change_count', 'last_50_day_40th_to_50th_pct_volume_change_count', 
                 'last_50_day_50th_to_60th_pct_volume_change_count', 'last_50_day_50th_pct_volume_change_count', 
                 'last_50_day_80th_pct_volume_change_count', 'last_50_day_40th_to_50th_pct_spread_count', 
                 'last_50_day_50th_to_60th_pct_spread_count', 'last_50_day_60th_to_70th_pct_spread_count', 
                 'last_50_day_70th_to_80th_pct_spread_count', 'last_50_day_80th_to_90th_pct_spread_count', 
                 'last_50_day_40th_pct_spread_count', 'last_50_day_50th_pct_spread_count', 'last_50_day_70th_pct_spread_count', 
                 'last_50_day_90th_pct_spread_count', 'rsi_100', 'last_100_day_60th_to_70th_pct_price_change_count', 
                 'last_100_day_80th_to_90th_pct_price_change_count', 'last_100_day_90th_to_95th_pct_price_change_count', 
                 'last_100_day_40th_pct_price_change_count', 'last_100_day_50th_pct_price_change_count', 
                 'last_100_day_60th_pct_price_change_count', 'last_100_day_80th_pct_price_change_count', 
                 'last_100_day_40th_to_50th_pct_volume_change_count', 'last_100_day_50th_to_60th_pct_volume_change_count', 
                 'last_100_day_60th_to_70th_pct_volume_change_count', 'last_100_day_70th_to_80th_pct_volume_change_count',
                 'last_100_day_80th_to_90th_pct_volume_change_count', 'last_100_day_40th_pct_volume_change_count', 
                 'last_100_day_60th_pct_volume_change_count', 'last_100_day_80th_pct_volume_change_count', 
                 'last_100_day_90th_pct_volume_change_count', 'last_100_day_40th_to_50th_pct_spread_count', 
                 'last_100_day_50th_to_60th_pct_spread_count', 'last_100_day_60th_to_70th_pct_spread_count', 
                 'last_100_day_70th_to_80th_pct_spread_count', 'last_100_day_90th_to_95th_pct_spread_count', 
                 'last_100_day_40th_pct_spread_count', 'last_100_day_50th_pct_spread_count', 'last_100_day_60th_pct_spread_count', 
                 'last_100_day_80th_pct_spread_count', 'last_100_day_90th_pct_spread_count']

cols_for_hopsworks = ['date', 'open', 'high', 'low', 'close', 'volume', 'volume_eth', 'market_cap']

eth_ohlc = eth_ohlc[cols_for_hopsworks + ['tmw_1_0_percent_increase_binary', 'tmw_percent_increase_to_avg_high_low'] + predictors_96]

eth_ohlc

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,1,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,2024-05-08,3013.0000,3036.0000,2944.0000,2975.0000,1.020000e+10,3400618.0,3.675000e+11,1,0.023697,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3079,2024-05-09,2977.0000,3055.0000,2958.0000,3036.0000,9.600000e+09,3190231.0,3.670000e+11,0,-0.018445,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3080,2024-05-10,3035.0000,3050.0000,2889.0000,2910.0000,1.060000e+10,3557434.0,3.650000e+11,0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.0000,2935.0000,2894.0000,2911.0000,5.900000e+09,2033385.0,3.569000e+11,1,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0


In [14]:
upload_data(eth_ohlc, 'eth_ohlc_transformed', 2)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/711829/fs/707652/fg/809321


Uploading Dataframe: 100.00% |██████████| Rows 3083/3083 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: eth_ohlc_transformed_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/711829/jobs/named/eth_ohlc_transformed_2_offline_fg_materialization/executions


### Pull from the hopsworks to check if the df is fine - theres a several minute lag time between uploading this data and being able to pull it...

In [17]:
hopsworks_eth_ohlc = pull_data('eth_ohlc_transformed', 2, 'eth_ohlc_transformed_view', 2)
hopsworks_eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/711829
Connected. Call `.close()` to terminate connection gracefully.
Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/711829/fs/707652/fv/eth_ohlc_transformed_view/version/2
Finished: Reading data from Hopsworks, using ArrowFlight (1.06s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_1_0_percent_increase_binary,tmw_percent_increase_to_avg_high_low,...,last_100_day_40th_to_50th_pct_spread_count,last_100_day_50th_to_60th_pct_spread_count,last_100_day_60th_to_70th_pct_spread_count,last_100_day_70th_to_80th_pct_spread_count,last_100_day_90th_to_95th_pct_spread_count,last_100_day_40th_pct_spread_count,last_100_day_50th_pct_spread_count,last_100_day_60th_pct_spread_count,last_100_day_80th_pct_spread_count,last_100_day_90th_pct_spread_count
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,1.0,0.033594,...,8.0,7.0,6.0,18.0,18.0,94.0,86.0,79.0,55.0,46.0
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.0,0.097955,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,45.0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,0.0,-0.009901,...,8.0,8.0,6.0,18.0,18.0,94.0,86.0,78.0,54.0,44.0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.0,-0.007323,...,8.0,8.0,7.0,18.0,17.0,94.0,86.0,78.0,53.0,43.0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.0,-0.015908,...,8.0,8.0,8.0,18.0,17.0,94.0,86.0,78.0,52.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,2024-05-08,3013.0000,3036.0000,2944.0000,2975.0000,1.020000e+10,3400618.0,3.675000e+11,1.0,0.023697,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3079,2024-05-09,2977.0000,3055.0000,2958.0000,3036.0000,9.600000e+09,3190231.0,3.670000e+11,0.0,-0.018445,...,14.0,8.0,10.0,8.0,3.0,47.0,33.0,25.0,7.0,3.0
3080,2024-05-10,3035.0000,3050.0000,2889.0000,2910.0000,1.060000e+10,3557434.0,3.650000e+11,0.0,0.004467,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
3081,2024-05-11,2909.0000,2935.0000,2894.0000,2911.0000,5.900000e+09,2033385.0,3.569000e+11,1.0,0.010306,...,14.0,9.0,10.0,8.0,3.0,48.0,34.0,25.0,7.0,3.0
